In [1]:
from __future__ import annotations

import re
import hashlib
from dataclasses import dataclass
from typing import Iterable, List, Tuple, Dict, Optional, Set

In [2]:
@dataclass(frozen=True)
class LSHEmbedding:
    """Output of LSH embedder."""
    signature: List[int]          # MinHash signature (length = num_hashes)
    band_keys: List[str]          # LSH bucket keys (length = num_bands)
    shingle_count: int            # number of unique shingles used

In [3]:
class LSHMinHashEmbedder:
    """
    MinHash + LSH banding embedder for near-duplicate detection / candidate generation.

    - Partial match support: character shingles (n-grams).
    - Vocabulary-free: uses hashing, no growing vocab table.
    - Persistent-friendly: `band_keys` are stable strings you can store (e.g., in Qdrant payload).

    Typical usage:
        emb = LSHMinHashEmbedder(shingle_size=5, num_hashes=128, bands=32, seed=42)
        out = emb.embed("raw log line ...")
        # candidate retrieval: find items that share any band_key
        # verification: recompute exact Jaccard of shingles (or other metric) vs candidates
    """

    # --- regexes for common log normalization ---
    _RE_UUID = re.compile(r"\b[0-9a-f]{8}-[0-9a-f]{4}-[1-5][0-9a-f]{3}-[89ab][0-9a-f]{3}-[0-9a-f]{12}\b", re.I)
    _RE_HEX  = re.compile(r"\b0x[0-9a-f]+\b", re.I)
    _RE_IPv4 = re.compile(r"\b(?:\d{1,3}\.){3}\d{1,3}\b")
    _RE_NUM  = re.compile(r"\b\d+\b")
    _RE_TS1  = re.compile(r"\b\d{4}-\d{2}-\d{2}[T ]\d{2}:\d{2}:\d{2}(?:\.\d+)?Z?\b", re.I)  # ISO-ish
    _RE_TS2  = re.compile(r"\b\d{2}:\d{2}:\d{2}(?:\.\d+)?\b")  # time only

    def __init__(
        self,
        shingle_size: int = 5,
        num_hashes: int = 128,
        bands: int = 32,
        seed: int = 42,
        normalize: bool = True,
        lowercase: bool = True,
        collapse_whitespace: bool = True,
        stop_short_lines: int = 0,
    ):
        """
        Args:
            shingle_size: character n-gram size (5–7 is typical for logs)
            num_hashes: MinHash signature length (64–256 typical)
            bands: number of LSH bands. Must divide num_hashes exactly.
            seed: controls determinism of the hash family
            normalize: apply log normalization (uuid/ip/nums/timestamps masking)
            lowercase: lowercase before shingling
            collapse_whitespace: replace consecutive whitespace with single space
            stop_short_lines: if >0 and normalized text shorter than this, returns empty shingles/signature behavior
        """
        if shingle_size <= 0:
            raise ValueError("shingle_size must be > 0")
        if num_hashes <= 0:
            raise ValueError("num_hashes must be > 0")
        if bands <= 0 or (num_hashes % bands != 0):
            raise ValueError("bands must be > 0 and must divide num_hashes exactly")

        self.shingle_size = shingle_size
        self.num_hashes = num_hashes
        self.bands = bands
        self.rows_per_band = num_hashes // bands

        self.seed = seed
        self.normalize_enabled = normalize
        self.lowercase = lowercase
        self.collapse_whitespace = collapse_whitespace
        self.stop_short_lines = stop_short_lines

        # A large prime > 2^32 for modular hashing
        self._prime = 4294967311  # near 2^32, prime
        self._max_hash = (1 << 32) - 1

        # Pre-generate hash function parameters (a_i, b_i)
        # h_i(x) = (a_i * x + b_i) mod prime
        self._a, self._b = self._make_hash_params(num_hashes, seed)

    # ------------------------- Public API -------------------------

    def embed(self, text: str) -> LSHEmbedding:
        """Compute MinHash signature + LSH band keys for a single log line."""
        norm = self._preprocess(text)
        if self.stop_short_lines and len(norm) < self.stop_short_lines:
            sig = [self._max_hash] * self.num_hashes
            bands = self._signature_to_band_keys(sig)
            return LSHEmbedding(signature=sig, band_keys=bands, shingle_count=0)

        shingles = self._shingle(norm)
        sig = self._minhash_signature(shingles)
        bands = self._signature_to_band_keys(sig)
        return LSHEmbedding(signature=sig, band_keys=bands, shingle_count=len(shingles))

    def shingles(self, text: str) -> Set[int]:
        """Return hashed shingles (useful for exact Jaccard verification)."""
        return self._shingle(self._preprocess(text))

    @staticmethod
    def jaccard(shingles_a: Set[int], shingles_b: Set[int]) -> float:
        """Exact Jaccard similarity between two shingle sets."""
        if not shingles_a and not shingles_b:
            return 1.0
        if not shingles_a or not shingles_b:
            return 0.0
        inter = len(shingles_a & shingles_b)
        union = len(shingles_a | shingles_b)
        return inter / union

    # ------------------------- Preprocess -------------------------

    def _preprocess(self, text: str) -> str:
        s = text
        if self.lowercase:
            s = s.lower()

        if self.normalize_enabled:
            s = self._RE_UUID.sub("<uuid>", s)
            s = self._RE_IPv4.sub("<ip>", s)
            s = self._RE_TS1.sub("<ts>", s)
            s = self._RE_TS2.sub("<time>", s)
            s = self._RE_HEX.sub("<hex>", s)
            s = self._RE_NUM.sub("<num>", s)

        if self.collapse_whitespace:
            s = re.sub(r"\s+", " ", s).strip()
        return s

    # ------------------------- Shingling -------------------------

    def _shingle(self, s: str) -> Set[int]:
        """
        Character n-gram shingling -> set of 32-bit ints (stable).
        Using a stable hash (blake2b) for shingles to avoid Python hash randomization.
        """
        n = self.shingle_size
        if len(s) < n:
            return set()

        out: Set[int] = set()
        # sliding window
        for i in range(0, len(s) - n + 1):
            gram = s[i : i + n]
            out.add(self._stable_u32(gram))
        return out

    # ------------------------- MinHash -------------------------

    def _minhash_signature(self, shingles: Set[int]) -> List[int]:
        """
        Compute MinHash signature over hashed shingles.
        If shingles empty: return max_hash vector so that it won't spuriously match.
        """
        if not shingles:
            return [self._max_hash] * self.num_hashes

        sig = [self._max_hash] * self.num_hashes
        p = self._prime

        # For each shingle x, update all hash functions:
        # sig[i] = min(sig[i], (a[i]*x + b[i]) % p)
        # (This is O(num_hashes * num_shingles). For very long lines, consider sampling shingles.)
        for x in shingles:
            for i in range(self.num_hashes):
                hx = (self._a[i] * x + self._b[i]) % p
                if hx < sig[i]:
                    sig[i] = hx

        # Convert modulo prime values into 32-bit range for compactness
        # (still deterministic; helps if you want to pack)
        return [v & self._max_hash for v in sig]

    def _signature_to_band_keys(self, sig: List[int]) -> List[str]:
        """
        Convert signature to LSH band keys.
        Band key is a stable string: "lsh:{band_index}:{hex_digest}".
        """
        keys: List[str] = []
        r = self.rows_per_band
        for b in range(self.bands):
            chunk = sig[b * r : (b + 1) * r]
            # Stable digest of the chunk
            digest = hashlib.blake2b(
                (",".join(map(str, chunk))).encode("utf-8"),
                digest_size=8  # 64-bit digest -> short key
            ).hexdigest()
            keys.append(f"lsh:{b}:{digest}")
        return keys

    # ------------------------- Hash utilities -------------------------

    @staticmethod
    def _make_hash_params(k: int, seed: int) -> Tuple[List[int], List[int]]:
        """
        Deterministically generate (a_i, b_i) pairs for k hash functions.
        We derive them from blake2b(seed||i) so results are stable across runs.
        """
        a: List[int] = []
        b: List[int] = []
        for i in range(k):
            h = hashlib.blake2b(f"{seed}:{i}".encode("utf-8"), digest_size=16).digest()
            # 64-bit a and b, then clamp into [1, prime-1] / [0, prime-1]
            ai = int.from_bytes(h[:8], "little") | 1  # make it odd / non-zero-ish
            bi = int.from_bytes(h[8:], "little")
            a.append(ai)
            b.append(bi)
        return a, b

    @staticmethod
    def _stable_u32(s: str) -> int:
        """Stable 32-bit unsigned hash for a string."""
        # blake2b is fast and stable; digest_size=4 gives 32-bit
        return int.from_bytes(hashlib.blake2b(s.encode("utf-8"), digest_size=4).digest(), "little")

In [4]:
emb = LSHMinHashEmbedder(shingle_size=5, num_hashes=128, bands=32, seed=123)

a = "ERROR 2026-01-13T12:00:01Z user_id=123e4567-e89b-12d3-a456-426614174000 request took 153ms"
b = "ERROR 2026-01-13T12:00:09Z user_id=223e4567-e89b-12d3-a456-426614174999 request took 149ms"
c = "INFO 2026-01-13T12:00:09Z user_id=223e4567-e89b-12d3-a456-426614174999 passed user"

ea = emb.embed(a)
eb = emb.embed(b)
ec = emb.embed(c)

# Candidate check: share any band?
overlap_ab = set(ea.band_keys) & set(eb.band_keys)
overlap_ac = set(ea.band_keys) & set(ec.band_keys)
print("A vs B band overlap:", len(overlap_ab), "out of", emb.bands)
print("A vs C band overlap:", len(overlap_ac), "out of", emb.bands)

# Verify exact Jaccard on shingles
ja = emb.shingles(a)
jb = emb.shingles(b)
jc = emb.shingles(c)
print("A vs B jaccard:", LSHMinHashEmbedder.jaccard(ja, jb))
print("A vs C jaccard:", LSHMinHashEmbedder.jaccard(ja, jc))

A vs B band overlap: 11 out of 32
A vs C band overlap: 1 out of 32
A vs B jaccard: 0.8181818181818182
A vs C jaccard: 0.3148148148148148


In [5]:
ea.band_keys

['lsh:0:5b12bfbeb0ade072',
 'lsh:1:e788d2732f29c5f9',
 'lsh:2:ea56b5865e440b9d',
 'lsh:3:a7f5210ee8333ff9',
 'lsh:4:61e6122b314304a9',
 'lsh:5:a24c3068b732600c',
 'lsh:6:ddc276ec409d0015',
 'lsh:7:f723b4ded7fd6af3',
 'lsh:8:4aa4d7d63d232caf',
 'lsh:9:b11b60769cc85d96',
 'lsh:10:d65429182bc472c8',
 'lsh:11:71328dd07180a8aa',
 'lsh:12:fa688b12e812ff2b',
 'lsh:13:f9240a08744e9750',
 'lsh:14:c4b912d14465d9fd',
 'lsh:15:d37011a6c7a52734',
 'lsh:16:a40884bec6641e62',
 'lsh:17:2f9e86428459eeb3',
 'lsh:18:c9626c2fcdefd5b9',
 'lsh:19:46f0ed02f7833081',
 'lsh:20:61c76801151e84db',
 'lsh:21:6ec7178c726b8bbc',
 'lsh:22:8d01b1c5f517a04b',
 'lsh:23:b3bcb51093d30cdd',
 'lsh:24:0b3833e4bd553639',
 'lsh:25:e4f908130211eacf',
 'lsh:26:4ccb02a86ccd5642',
 'lsh:27:7acefe4a27a71447',
 'lsh:28:314c795d670b9caf',
 'lsh:29:89e15f43c55a6476',
 'lsh:30:f086277b30e57f5e',
 'lsh:31:e6c02dcfbd6016a7']

## QdrantVectorStore

In [7]:
from qdrant_client import QdrantClient

In [ ]:
class VectorStore:
    def __init__(
        self,
        collection_name,
        url="http://localhost:6333"
    ):
        self.collection_name = collection_name
        self.url = url
        self.client = QdrantClient(url="http://localhost:6333")

    def create_collection_if_not_exists(self, vector_size: int, distance: str = "Cosine"):
        if not self.client.collection_exists(collection_name=self.collection_name):
            self.client.create_collection(
                collection_name=self.collection_name
            )
    
    def delete_collection_if_exists(self):
        if self.client.collection_exists(collection_name=self.collection_name):
            self.client.delete_collection(collection_name=self.collection_name)
    
    def close(self):
        self.client.close()

In [ ]:
vector_store = VectorStore(collection_name="test_collection")